In [ ]:
!pip install -qq -U diffusers transformers ftfy gradio accelerate
!pip install omegaconf
import inspect
from typing import List, Optional, Union
#@markdown 运行完点重启
import numpy as np
import torch

import PIL
import gradio as gr
from diffusers import StableDiffusionInpaintPipeline

Link = '' #@param {type:"string"}
!mkdir 'model'
name = 'dreamshaper_8Inpainting.safetensors' #@param {type:"string"}
import os
if not os.path.exists('./model/'+name):
  !wget $Link -P 'model' --content-disposition

device = "cuda"
pipe = StableDiffusionInpaintPipeline.from_single_file(
    '/content/model/dreamshaper_8Inpainting.safetensors',
    torch_dtype=torch.float16,
    load_safety_checker =False,
    local_files_only=True
).to(device)

In [ ]:
#@markdown RUN
MAX_W = 400 #@param {type:"number"}
MAX_H = 1200 #@param {type:"number"}
def predict2(dict, prompt,negative_prompt):
  w,h = dict['image'].size
  while w*h > MAX_W*MAX_H:
    w = int(w/1.1)
    h = int(h/1.1)
  while w%8 !=0:
    w += 1
  while h%8 != 0:
    h += 1
  image =  dict['image'].convert("RGB").resize((w, h))
  mask_image = dict['mask'].convert("RGB").resize((w, h))
  images = pipe(prompt=prompt, negative_prompt=negative_prompt,image=image, mask_image=mask_image,height=h,width=w).images
  return(images[0])
gr.Interface(
    predict2,
    title = 'Stable Diffusion In-Painting',
    inputs=[
        gr.Image(source = 'upload', tool = 'sketch', type = 'pil'),
        gr.Textbox(label = 'prompt'),
        gr.Textbox(label='negative_prompt')
    ],
    outputs=[
        gr.Image()
    ],
    layout="vertical"

).launch(debug=True)

In [ ]:
#@markdown RUN
MAX_W = 400 #@param {type:"number"}
MAX_H = 1200 #@param {type:"number"}
def predict2(dict, prompt,negative_prompt):
  w,h = dict['image'].size
  while w*h > MAX_W*MAX_H:
    w = int(w/1.1)
    h = int(h/1.1)
  while w%8 !=0:
    w += 1
  while h%8 != 0:
    h += 1
  image =  dict['image'].convert("RGB").resize((w, h))
  mask_image = dict['mask'].convert("RGB").resize((w, h))
  images = pipe(prompt=prompt, negative_prompt=negative_prompt,image=image, mask_image=mask_image,height=h,width=w).images
  return(images[0])
gr.Interface(
    predict2,
    title = 'Stable Diffusion In-Painting',
    inputs=[
        gr.Image(source = 'upload', tool = 'sketch', type = 'pil',width=1000,height=1000),
        gr.Textbox(label = 'prompt'),
        gr.Textbox(label='negative_prompt')
    ],
    outputs=[
        gr.Image()
    ],
    layout="vertical"

).launch(debug=True)